In [ ]:
import numpy as np

In [ ]:
import pandas as pd


df = pd.read_csv(r'././data/aida_competition.csv')


df

In [ ]:

df['Remarks_norm'] = df['Remarks'].apply(lambda x: str(x).lower())
cond_list = [df['Remarks_norm'].str.contains('|'.join(['bo', 'blackout']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['early']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['pull']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['record', 'national', 'nr']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['grab']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['airway']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['no tag']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['dqsp']), case=False)
             ]
choice_list = ['DQBO', 'EARLY TURN', 'DQPULL', 'NR', 'GRAB', 'DQAIRWAYS', 'NO TAG', 'DQSP']
df['Remarks_norm'] = np.select(cond_list, choice_list, default='')

print(len(df['Remarks'].unique()))
remarks_df = sorted([x for x in df['Remarks_norm'].unique()])
print(len(remarks_df))
df[['Remarks','Remarks_norm']].to_csv('remarks.csv')

In [ ]:
df['Category Event'] = df['Title Event'].apply(lambda x: x.lower())
cond_list = [df['Category Event'].str.contains('|'.join(['vertical blue']), case=False),
             df['Category Event'].str.contains('|'.join(['nac', 'nice abyss contest']), case=False),
             df['Category Event'].str.contains('|'.join(['world']), case=False),
             df['Category Event'].str.contains('|'.join(['panglao']), case=False)
             ]
choice_list = ['VB', 'NAC', 'World Championship', 'Panglao']
df['Category Event'] = np.select(cond_list, choice_list, default='other')

print(len(df['Title Event'].unique()))
remarks_df = sorted([x for x in df['Title Event'].unique()])
print(len(remarks_df))
df[['Title Event','Category Event']].to_csv('events.csv')
pd.DataFrame(remarks_df).to_csv('cat_events.csv')
# pd.DataFrame(sorted([x for x in df['Category Event'].unique()])).to_csv('cat_events.csv')

In [ ]:
df['Month'] = pd.to_datetime(df['Day']).dt.month 

labels=['Winter', 'Spring', 'Summer', 'Fall']
doy = pd.to_datetime(df['Day']).dt.dayofyear
df['Season'] = pd.cut(doy + 11 - 366*(doy > 355), bins=[0, 91, 183, 275, 366], labels=labels)

df['Experience dive'] = df.groupby('Diver').cumcount() + 1
df['Experience discipline'] = df.groupby(['Diver', 'Discipline']).cumcount() + 1
# df['Experience event'] = df.groupby(['Diver', 'Title Event']).cumcount() + 1
df[df['Diver'] == 'Clementine Marie (FRA)']
# df['Diver'] == 'Abdel Alouach'
# df
# df['Remarks_norm'] = df['Remarks'].apply(lambda x: x.lower())
# df['AP'] = df['AP'].apply(lambda x: x.split()[0])
df.to_csv('aida_competition_clean.csv', index=False)

In [ ]:
df.head()

In [ ]:
cond = 'Remark_norm' == 'DBQO'
list_col = ['Experience dive'] #'Discipline', 'AP', 'Gender', 

for col in list_col :
    count_discipline = df[col].value_counts() 
    count_discipline_cond = df[df['Remarks_norm'] == 'DQBO'][col].value_counts() 
    total = count_discipline*100 /df.shape[0]
    prop = count_discipline_cond * 100/ count_discipline
    moy_exclu_max = prop[prop != prop.iloc[prop.argmax()]].mean()
    print(count_discipline*100 /df.shape[0])
    print(prop.iloc[prop.argmax()])
    print(moy_exclu_max)
    print(prop/moy_exclu_max)


In [ ]:
# df.groupby(['Diver','Discipline']).agg(['count'])

In [ ]:
gk = df.groupby('Diver')

In [ ]:
first_compet = gk['Day'].first()
last_compet = gk['Day'].last()
nb_dives = gk['Title Event'].count()
nb_compets = gk['Title Event'].nunique()


In [139]:
# df.groupby(['Diver', 'Remarks_norm']).count()['Remarks_norm']
# gk.apply(lambda x: x[x['Remarks_norm'] == 'DQBO'].count())
nb_bo=df.groupby('Diver')['Remarks_norm'].apply(lambda x: (x=='DQBO').sum()).reset_index(name='count')
nb_red=df.groupby('Diver')['Card'].apply(lambda x: (x=='RED').sum()).reset_index(name='count')


In [ ]:
ap_compet = gk['AP'].min(), gk['AP'].max(), gk['AP'].mean()

In [131]:
nb_bo.values[:, 1]

array([4, 0, 0, ..., 0, 0, 0], dtype=object)

In [143]:
df_diver = pd.DataFrame({ 'diver' : sorted(df['Diver'].unique()),
                          'first_compet':first_compet.values,
                          'last_compet':last_compet.values,
                          'nb_compet': nb_compets.values,
                          'nb_bo' : nb_bo.values[:, 1],
                          'nb_red' : nb_red.values[:, 1],
                          'nb_dives' : gk['Experience dive'].last(),
                          'percent_bo' :  nb_bo.values[:, 1] *100 /gk['Experience dive'].last()

                          })#, last_compet, *ap_compet])
# df_reversed = df_diver[::-1]

In [144]:
df_diver[df_diver['diver']=='Clementine Marie (FRA)']

,diver,first_compet,last_compet,nb_compet,nb_bo,nb_red,nb_dives
Diver,,,,,,,
Clementine Marie (FRA),Clementine Marie (FRA),2021-07-11,2023-07-23,6,0,1,21


In [138]:
df[df['Diver']=='Clementine Marie (FRA)']

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event,Month,Season,Experience dive,Experience discipline,Remarks_norm
20572,12,Clementine Marie (FRA),F,CWTB,1.0,15:50,45,45 m,WHITE,45.0,Ok,AIDA Freediving World CUP,Depth Competition,2021-07-11,World Championship,7,Summer,1,1,
20587,13,Clementine Marie (FRA),F,CWT,1.0,16:00,51,51 m,WHITE,51.0,Ok,AIDA Freediving World CUP,Depth Competition,2021-07-12,World Championship,7,Summer,2,1,
20602,13,Clementine Marie (FRA),F,FIM,1.0,16:30,50,50 m,WHITE,50.0,Ok,AIDA Freediving World CUP,Depth Competition,2021-07-14,World Championship,7,Summer,3,1,
20617,13,Clementine Marie (FRA),F,CNF,1.0,16:20,36,36 m,WHITE,36.0,Ok,AIDA Freediving World CUP,Depth Competition,2021-07-15,World Championship,7,Summer,4,1,
20855,22,Clementine Marie (FRA),F,CWTB,1.0,12:39,41,41 m,WHITE,41.0,Ok,NAC AIDA 2021,Depth Competition,2021-06-20,NAC,6,Spring,5,2,
21062,8,Clementine Marie (FRA),F,FIM,1.0,09:40,55,55 m,WHITE,55.0,Ok,Depth World Championship,Depth Competition,2021-09-24,World Championship,9,Fall,6,2,
21147,5,Clementine Marie (FRA),F,CNF,NaN,00:00,45,45 m,WHITE,45.0,Ok,Depth World Championship,Depth Competition,2021-09-26,World Championship,9,Fall,7,2,
21216,26,Clementine Marie (FRA),F,CWT,2.0,11:00,55,55 m,WHITE,55.0,Ok,Depth World Championship,Depth Competition,2021-09-28,World Championship,9,Fall,8,2,
21522,9,Clementine Marie (FRA),F,CWTB,NaN,00:00,40,40 m,WHITE,40.0,Ok,AIDA WORLD CHAMPIONSHIPS PRE-COMPETITION ANDIK...,Depth Competition,2021-09-12,World Championship,9,Summer,9,3,
21549,13,Clementine Marie (FRA),F,CNF,NaN,00:00,40,40 m,WHITE,40.0,Ok,AIDA WORLD CHAMPIONSHIPS PRE-COMPETITION ANDIK...,Depth Competition,2021-09-14,World Championship,9,Summer,10,3,


In [145]:
df_diver.to_csv('diver_stat.csv', index=False)